# Altegrad Experiments

In [1]:
import numpy as np

#embeddings = np.load('D:\Scolaire\Code\Python\Machine Learning\Kaggle Challenges M2\Altegrad - Graphs HAN\data\embeddings.npy')

In [5]:
#embeddings.shape

(1685895, 13)

# Training NN

In [2]:
#!python main_baseline.py

# Inference using trained NN

In [3]:
#!python read_results_predict.py

# Kaggle submissions

In [2]:
import pandas as pd
import os

def from_txt_to_csv(file_name, folder_name):
    """
    Transform the output of the read_results_predict.py in a proper Kaggle Submission, i.e : a well formated csv file
    
    inputs : 
    - file_name is the name (string) of the txt file generated by the read_results_predict.py (without the .txt extension)
    - folder_name is the desired or existing name (string) of the folder where the submission will be stored into 
    
    output : None
    
    """
    # change me !
    path_root = 'D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/'
    path_file = path_root + file_name + ".txt"
    path_folder = path_root + folder_name
    # create a directory if it doesn't exist yet
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)
      
    df = pd.read_csv(path_file)
    df.to_csv(path_folder + "/" + file_name + ".csv", index=False)
    print("Submission saved in '{}'".format(path_folder))
    

In [3]:
from_txt_to_csv('predictions_han', 'exp_layers')

Submission saved in 'D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/exp_layers'


deep walk nous donne un fichier .embeddings contenant un en tête (?)
et une liste de la forme numéro_du_noeud - vecteur representant l'embedding

Il faut trouver un moyen de calculer ça pour tous les fichiers et de stocker le résultat final dans une grande matrice où numero du noeud = ligne matrice embedding


In [ ]:
# import os
# for i in range(0, 93719): 
#         input_dest = "D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/edge_lists/" + str(i) + ".txt" 
#         output_dest = "./embeddings/emb" + str(i) + ".embeddings" 
#         os.system("deepwalk --format edgelist --workers 20 --input "+ input_dest +" --output " + output_dest)



In [1]:
!python ./experiments/preprocessing_baseline.py

0
9372
18744
28116
37488
46860
56232
65604
74976
84348
documents generated
document array shape: (93719, 100, 21)
documents saved
everything done in 932.11


# Same architecture but far bigger document.npy

In [8]:
!python ./experiments/adding_layers.py


Epoch 00001: val_loss improved from inf to 0.33995, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 00002: val_loss improved from 0.33995 to 0.31840, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 00003: val_loss improved from 0.31840 to 0.26294, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 00004: val_loss improved from 0.26294 to 0.25565, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 00005: val_loss improved from 0.25565 to 0.24104, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 00006: val_loss did not improve from 0.24104

Epoch 00007: val_loss improved from 0.24104 to 0.22166, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_0

Epoch 000

Using TensorFlow backend.


In [1]:
!python ./read_results_predict.py

Using TensorFlow backend.


Conclusion : baseline architecture (linear activation) + baseline embeddings

target 0 : ok
les autres à tuner

target 2 is overfitting... 

Code to retrain one specific target

In [1]:
import sys
import json
import numpy as np

from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.models import Model
from keras.layers import Input, Embedding, Dropout, Bidirectional, GRU, CuDNNGRU, TimeDistributed, Dense

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# = = = = = = = = = = = = = = =


is_GPU = False
save_weights = True
save_history = False
np.random.seed(1997)

path_root = 'D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad'
path_to_code = path_root + '/code/'
path_to_data = path_root + '/data/'

sys.path.insert(0, path_to_code)

# = = = = = = = = = = = = = = =

from AttentionWithContext import AttentionWithContext
from make_model import make_model


# = = = = = hyper-parameters = = = = =

tgt = 2
n_units = 80
drop_rate = 0.5
batch_size = 96
nb_epochs = 15
my_optimizer = 'adam'
my_patience = 4

# = = = = = data loading = = = = =

docs = np.load(path_to_data + 'documents.npy')
embeddings = np.load(path_to_data + 'embeddings.npy')

with open(path_to_data + 'train_idxs.txt', 'r') as file:
    train_idxs = file.read().splitlines()
    
train_idxs = [int(elt) for elt in train_idxs]
    
# create validation set

idxs_select_train = np.random.choice(range(len(train_idxs)), size=int(len(train_idxs) * 0.80), replace=False)
idxs_select_val = np.setdiff1d(range(len(train_idxs)), idxs_select_train)

train_idxs_new = [train_idxs[elt] for elt in idxs_select_train]
val_idxs = [train_idxs[elt] for elt in idxs_select_val]

docs_train = docs[train_idxs_new, :, :]
docs_val = docs[val_idxs, :, :]


with open(path_to_data + 'targets/train/target_' + str(tgt) + '.txt', 'r') as file:
    target = file.read().splitlines()
    
target_train = np.array([target[elt] for elt in idxs_select_train]).astype('float')
target_val = np.array([target[elt] for elt in idxs_select_val]).astype('float')

# target 2 is over fitting so maybe try more units and dropout -> more expressivity    

model = make_model(n_units, drop_rate, embeddings, docs_train, is_GPU)
 
model.compile(loss='mean_squared_error',
                  optimizer=my_optimizer,
                  metrics=['mse'])

# = = = = = training = = = = =

early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=my_patience,
                                   mode='min')

    # save model corresponding to best epoch
checkpointer = ModelCheckpoint(filepath=path_to_data + 'model_' + str(tgt), 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True)

if save_weights:
    my_callbacks = [early_stopping, checkpointer]
else:
    my_callbacks = [early_stopping]

model.fit(docs_train, 
              target_train,
              batch_size = batch_size,
              epochs = nb_epochs,
              validation_data = (docs_val, target_val),
              callbacks = my_callbacks)
    

if save_history:
    hist = model.history.history
    with open(path_to_data + 'model_history_' + str(tgt) + '.json', 'w') as file:
        json.dump(hist, file, sort_keys=False, indent=4)

print('* * * * * * * target', tgt, 'done * * * * * * *')    
    

Using TensorFlow backend.


Train on 59980 samples, validate on 14995 samples
Epoch 1/15
59980/59980 [==============================] - 189s 3ms/step - loss: 0.6941 - mean_squared_error: 0.6941 - val_loss: 0.5619 - val_mean_squared_error: 0.5619

Epoch 00001: val_loss improved from inf to 0.56191, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_2
Epoch 2/15
59980/59980 [==============================] - 185s 3ms/step - loss: 0.5972 - mean_squared_error: 0.5972 - val_loss: 0.5367 - val_mean_squared_error: 0.5367

Epoch 00002: val_loss improved from 0.56191 to 0.53673, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/Altegrad/data/model_2
Epoch 3/15
59980/59980 [==============================] - 193s 3ms/step - loss: 0.5698 - mean_squared_error: 0.5698 - val_loss: 0.5761 - val_mean_squared_error: 0.5761

Epoch 00003: val_loss did not improve from 0.53673
Epoch 4/15
59980/59980 [==============================] - 197s 3ms/step - loss: 0.54